Ovadia Sutton
AIM 5001 Project 1
Text Processing and Regular Expressions

Import libraries

In [466]:
import re
import pandas as pd
import numpy as np
import statistics as st

Declare a text variable with text block provided

In [467]:
#Decalre text as one long string 
text = "555-1239Dr. Anthony Fauci(636) 555-0113Hollingdorp, Donnatella555-6542Fitzgerald, F. Scott555 8904Rev. Martin Luther King636-555-3226Snodgrass, Theodore5553642Carlamina Scarfoni" 

#Task 1:
Extract the names of each individual from the unformatted text string shown above and store them in a vector of some sort.

We collect strings that contain any letters both uppercase and lowercase and allow the possibility of punctuation such as  , . or space. We demand 2 plus matches to avoid collecting the single spaces. 

In [468]:
#Declare a variable 'names' which finds strings of words and related punctuation and spacing - therefore extracting each full name as a separate entry in our vector
names = re.findall('([A-Za-z?,?.?\s?]{2,})', text)

#print results
print(names)

['Dr. Anthony Fauci', 'Hollingdorp, Donnatella', 'Fitzgerald, F. Scott', 'Rev. Martin Luther King', 'Snodgrass, Theodore', 'Carlamina Scarfoni']


#Task 2a:
Use your regex skills to rearrange the vector so that all elements conform to the standard “firstname lastname”, preserving any titles (e.g., “Rev.”, “Dr.”, etc) or middle/second names. 

Here I create a new vector. We loop through the names vector and identify if a name is followed by a ','. This indicates a last name preceding the first name. We use the sub function to swap the order of the two groups

In [469]:
#Declare new list
firstlast = []

#Loop through names and identify last names preceding first names based on ','. then swap first and last name
for i in names:
    firstlast.append(re.sub('^([^,]*), (.*)', "\g<2> \g<1>", i))

#print reults
print(firstlast)  

['Dr. Anthony Fauci', 'Donnatella Hollingdorp', 'F. Scott Fitzgerald', 'Rev. Martin Luther King', 'Theodore Snodgrass', 'Carlamina Scarfoni']


#Task 2b:
Using your regex skills, construct a logical vector indicating whether a character has a title (i.e., Rev. and Dr.). 

Again we create a new vector and that loops through our new list of firstlast names.
If the first letters encountered are more than two letters (avoiding single initital) and are followed by a '.', we deem it a title and add full name to vector. 

In [470]:
#Create a new list
Title = []

#Loop through names and identify titles by seeing two or more letters followed by a '.'
for i in firstlast:
    Title.append(re.findall('([A-Za-z]{2,}[\.][A-Za-z\s]+)', i))

#Print results
print(Title)

[['Dr. Anthony Fauci'], [], [], ['Rev. Martin Luther King'], [], []]


We use a pandas series to extract the logical version of this in a table
We can also use the str.contains to see just true or false, but we were prompted to use extract by Jupyter
Here we also extract just the title in a separate column

In [471]:
#Declare firstlast in pd series
firstlast = pd.Series(firstlast)

#Extract results based on the same regex as above
firstlast.str.extract('(([A-Za-z]{2,}[\.])[A-Za-z\s]+)')

,0,1
0,Dr. Anthony Fauci,Dr.
1,NaN,NaN
2,NaN,NaN
3,Rev. Martin Luther King,Rev.
4,NaN,NaN
5,NaN,NaN


#Task 2c:
Using your regex skills, construct a logical vector indicating whether a character has a middle/second name. 

Use pandas vectorized string functions to identify if person has a middle name
We do this by allowing for a first initial. (one capital letter) followed by two more words separated by a space, or 3 words without any '.' in them

*We note that we could have used the above approaches as well (new list or pandas extract function)

In [472]:
#Use pandas contians to find middle name (3 separate words or two preceded by a single initial)
firstlast.str.contains('(([A-Z\.]{2}|[A-Za-z]+)[\s][A-Za-z]+[\s][A-Za-z]+)')

C:\Users\OVADIA~1\AppData\Local\Temp/ipykernel_98928/3618682810.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  firstlast.str.contains('(([A-Z\.]{2}|[A-Za-z]+)[\s][A-Za-z]+[\s][A-Za-z]+)')


0    False
1    False
2     True
3     True
4    False
5    False
dtype: bool

#Task 3:
Consider the HTML string <title>+++BREAKING NEWS+++<title>. We would like to extract the first HTML tag (i.e., “<title>”). To do so we write the regular expression “<.+>”.  Explain why this fails and correct the expression. 

From https://docs.python.org/3/library/re.html
"
*?, +?, ??
The '*', '+', and '?' qualifiers are all greedy; they match as much text as possible. Sometimes this behaviour isn’t desired; if the RE <.*> is matched against '<a> b <c>', it will match the entire string, and not just '<a>'. Adding ? after the qualifier makes it perform the match in non-greedy or minimal fashion; as few characters as possible will be matched. Using the RE <.*?> will match only '<a>'.
"
The same is true here for the +. In short the + is more inclusive than we would like it to be and it matches the entire string. 

In [473]:
#Without the ? matches entire string 

#Declare string
news = '<title>+++BREAKING NEWS+++<title>'

#Perform search with regex in problem (the greedy +)
search = re.search('<.+>', news)

#Print results shows entire string is in search
print(search)

<re.Match object; span=(0, 33), match='<title>+++BREAKING NEWS+++<title>'>


And so we edit to +? giving the desired result

In [474]:
#With the ? only <title>

#Declare string
news = '<title>+++BREAKING NEWS+++<title>'

#Perform search with the non-greedy +?
search = re.search('<.+?>', news)

#Print resulst shows search returns as desired
print(search)

<re.Match object; span=(0, 7), match='<title>'>


Task 4:
Consider the string “(5-3)^2=5^2-2*5*3+3^2” conforms to the binomial theorem. We would like to extract the equation in its entirety from the string. To do so we write the regular expression “[^0-9=+*()]+”. Explain why this fails and correct the expression. 

The '^'  -  "Matches the start of the string" (https://docs.python.org/3/library/re.html)
As such, here it is not including the ^ as the "power of" sign rather the start of string 
As well, the '-' sign is only used here to express range from 0 to 9 and is not including the "subtraction" sign

In [475]:
#Declare binomial string
binomial = '(5-3)^2=5^2-2*5*3 +3^2'

#Search with above suggestion
search = re.search('[^0-9=+*()]+', binomial)

#Print results showing only beginning is observed
print(search)

<re.Match object; span=(2, 3), match='-'>


So we edit moving the '^' to the end as well as an extra '-' at the end

In [476]:
#Declare binomial string
binomial = '(5-3)^2=5^2-2*5*3 +3^2'

#Search with ^ and - at end
search = re.search('[0-9=+*()^-]+', binomial)

#Print results and show the entire equation as desired
print(search)

<re.Match object; span=(0, 17), match='(5-3)^2=5^2-2*5*3'>


Part 2

We read in the file for reference

In [477]:
#Set f to open file path
f = open(r"C:\Users\ovadia sutton\Downloads\Project1.txt")

#read f and print
print(f.read())

-----------------------------------------------------------------------------------------
 Pair | Player Name                     |Total|Round|Round|Round|Round|Round|Round|Round| 
 Num  | USCF ID / Rtg (Pre->Post)       | Pts |  1  |  2  |  3  |  4  |  5  |  6  |  7  | 
-----------------------------------------------------------------------------------------
    1 | GARY HUA                        |6.0  |W  39|W  21|W  18|W  14|W   7|D  12|D   4|
   ON | 15445895 / R: 1794   ->1817     |N:2  |W    |B    |W    |B    |W    |B    |W    |
-----------------------------------------------------------------------------------------
    2 | DAKSHESH DARURI                 |6.0  |W  63|W  58|L   4|W  17|W  16|W  20|W   7|
   MI | 14598900 / R: 1553   ->1663     |N:2  |B    |W    |B    |W    |B    |W    |B    |
-----------------------------------------------------------------------------------------
    3 | ADITYA BAJAJ                    |6.0  |L   8|W  61|W  25|W  21|W  11|W  13|W  12|
   MI | 

Store player names

Player names uniquely have at least two words and all in caps. We read in the file and collect all strings with at least one capital letter followed by a space followed by at least one capital letter followed by a space and then allow for the option of more words and spaces until we observe the '|'. We also allow "',.-" in the middle and last name. 

In [478]:
#Create empty list
PlayerNames = []

#Open file
with open(r"C:\Users\ovadia sutton\Downloads\Project1.txt") as f:
    #Loop through lines in file
   for line in f:
    #Append list finding two or more capital strings until the "|" corresponding to name
    PlayerNames.append(re.findall('[A-Z]{1,}\s[A-Z.,-]{1,}\s[A-Z-\s]+?\|', line))
    
#Eliminate empty strings
PlayerNames = list(filter(None, PlayerNames))

#Create list for clean entries
PlayerNamesFinal = []

#Strip the space and '|'
#As we have a list of list we use two loops
#Loop through list 1
for i in PlayerNames:
    #Take list within list
    for n in i:
        #Strip undesired elements
        PlayerNamesFinal.append(n.strip("  |"))

#Print length to confirm we have exactly 64 entries per number of players
print(len(PlayerNamesFinal))

#Print our list to cross-reference with text file 
print(PlayerNamesFinal)

64
['GARY HUA', 'DAKSHESH DARURI', 'ADITYA BAJAJ', 'PATRICK H SCHILLING', 'HANSHI ZUO', 'HANSEN SONG', 'GARY DEE SWATHELL', 'EZEKIEL HOUGHTON', 'STEFANO LEE', 'ANVIT RAO', 'CAMERON WILLIAM MC LEMAN', 'KENNETH J TACK', 'TORRANCE HENRY JR', 'BRADLEY SHAW', 'ZACHARY JAMES HOUGHTON', 'MIKE NIKITIN', 'RONALD GRZEGORCZYK', 'DAVID SUNDEEN', 'DIPANKAR ROY', 'JASON ZHENG', 'DINH DANG BUI', 'EUGENE L MCCLURE', 'ALAN BUI', 'MICHAEL R ALDRICH', 'LOREN SCHWIEBERT', 'MAX ZHU', 'GAURAV GIDWANI', 'SOFIA ADINA STANESCU-BELLU', 'CHIEDOZIE OKORIE', 'GEORGE AVERY JONES', 'RISHI SHETTY', 'JOSHUA PHILIP MATHEWS', 'JADE GE', 'MICHAEL JEFFERY THOMAS', 'JOSHUA DAVID LEE', 'SIDDHARTH JHA', 'AMIYATOSH PWNANANDAM', 'BRIAN LIU', 'JOEL R HENDON', 'FOREST ZHANG', 'KYLE WILLIAM MURPHY', 'JARED GE', 'ROBERT GLEN VASEY', 'JUSTIN D SCHILLING', 'DEREK YAN', 'JACOB ALEXANDER LAVALLEY', 'ERIC WRIGHT', 'DANIEL KHAIN', 'MICHAEL J MARTIN', 'SHIVAM JHA', 'TEJAS AYYAGARI', 'ETHAN GUO', 'JOSE C YBARRA', 'LARRY HODGE', 'ALEX KONG

Store player states 

Player states are uniquely two capital letters followed by a space and '|'

In [479]:
#Create empty list
PlayerStates = []

#Open file 
with open(r"C:\Users\ovadia sutton\Downloads\Project1.txt") as f:
    #Loop through the lines in file 
   for line in f:
    #Append list finding exactly two capital letters and space then "|" - corresponding to state 
    PlayerStates.append(re.findall('[A-Z]{2}\s\|', line))
    
#Eliminate empty strings
PlayerStates = list(filter(None, PlayerStates))

#Create list for clean entries
PlayerStatesFinal = []

#Strip the space and '|'
#As we have a list of list we use two loops
#Loop through list 1
for i in PlayerStates:
    #Take list within list
    for n in i:
        #Strip undesired elements
        PlayerStatesFinal.append(n.strip("  |"))
        

#Print length to confirm we have exactly 64 entries per number of players
print(len(PlayerStates1))

#Print list to cross-reference with text file 
print(PlayerStates1)

64
['ON', 'MI', 'MI', 'MI', 'MI', 'OH', 'MI', 'MI', 'ON', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'ON', 'MI', 'ON', 'MI', 'MI', 'ON', 'MI', 'MI', 'MI', 'ON', 'MI', 'ON', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'MI', 'ON', 'MI', 'MI', 'MI']


Store player score

Player scores are uniquely a digit followed by a '.' followed by a second digit

In [480]:
#Create empty list
PlayerScores = []

#Open file 
with open(r"C:\Users\ovadia sutton\Downloads\Project1.txt") as f:
   #Loop through the lines in file 
   for line in f:
    #Append list with digit followed by '.' followed by digit - correspnding to score
    PlayerScores.append(re.findall('\d\.\d', line))

#Eliminate empty strings
PlayerScores = list(filter(None, PlayerScores))

#Create list for clean entries
PlayerScoresFinal = []

#Strip the space and '|'
#As we have a list of list we use two loops
#Loop through list 1
for i in PlayerScores:
    #Take list within list
    for n in i:
        #Strip undesired elements
        PlayerScoresFinal.append(n.strip("[]"))

#Print length to confirm we have exactly 64 entries per number of players
print(len(PlayerScoresFinal))

#Print list to cross-reference with text file
print(PlayerScoresFinal)

64
['6.0', '6.0', '6.0', '5.5', '5.5', '5.0', '5.0', '5.0', '5.0', '5.0', '4.5', '4.5', '4.5', '4.5', '4.5', '4.0', '4.0', '4.0', '4.0', '4.0', '4.0', '4.0', '4.0', '4.0', '3.5', '3.5', '3.5', '3.5', '3.5', '3.5', '3.5', '3.5', '3.5', '3.5', '3.5', '3.5', '3.5', '3.0', '3.0', '3.0', '3.0', '3.0', '3.0', '3.0', '3.0', '3.0', '2.5', '2.5', '2.5', '2.5', '2.5', '2.5', '2.0', '2.0', '2.0', '2.0', '2.0', '2.0', '2.0', '1.5', '1.5', '1.0', '1.0', '1.0']


Store player  pre tournament ratings

Player's pre-tournament rating are uniquely "R:" followed by one or two spaces followed by a 3-4 digit rating.
(We may exclude the R: but should include the space to prevent collecting the post tournament rating)

*We note the interest to store this in a computable way and save for next step

In [481]:
#Create empty list
PlayerRatingPre = []

#Open file 
with open(r"C:\Users\ovadia sutton\Downloads\Project1.txt") as f:
    #Loop through the lines in file 
   for line in f:
    #Append list with digit followed by '.' followed by digit - correspnding to score
    PlayerRatingPre.append(re.findall('R:\s\s?\d+', line))
    
#Eliminate empty strings
PlayerRatingPre = list(filter(None, PlayerRatingPre))

#Create list for clean entries
PlayerRatingPreFinal = []

#Strip the space and '|'
#As we have a list of list we use two loops
#Loop through list 1
for i in PlayerRatingPre:
    #Take list within list
    for n in i:
        #Strip undesired elements
        PlayerRatingPreFinal.append(n.strip("R:  "))

#Print length to confirm we have exactly 64 entries per number of players
print(len(PlayerRatingPreFinal))

#Print list to cross-reference with text file
print(PlayerRatingPreFinal)

64
['1794', '1553', '1384', '1716', '1655', '1686', '1649', '1641', '1411', '1365', '1712', '1663', '1666', '1610', '1220', '1604', '1629', '1600', '1564', '1595', '1563', '1555', '1363', '1229', '1745', '1579', '1552', '1507', '1602', '1522', '1494', '1441', '1449', '1399', '1438', '1355', '980', '1423', '1436', '1348', '1403', '1332', '1283', '1199', '1242', '377', '1362', '1382', '1291', '1056', '1011', '935', '1393', '1270', '1186', '1153', '1092', '917', '853', '967', '955', '1530', '1175', '1163']


Store average opponent rating

Approach: 
1. Store our unique player IDs. 
2. Create a dictionary with keys as unique ID and values as Pre-tournament rating
3. For each player we store their opponent's ID of each game
4. We use this to reference the dictionary and thier ratings
5. Take an average of their ratings and return

Create dictionary of IDs as key and Ratings as value

In [482]:
#Instead of pulling ID's from data we create a list from 1-64 and call it ID
ID = list(range(1, 65))

#Store ratings as int values 
PreRatingInt = [int(float(x)) for x in PlayerRatingPreFinal]

#Create dictionary with ID as key and PreRatingInt as value
IDRating = dict(zip(ID, PreRatingInt))

#Print dictionary
print(IDRating)

{1: 1794, 2: 1553, 3: 1384, 4: 1716, 5: 1655, 6: 1686, 7: 1649, 8: 1641, 9: 1411, 10: 1365, 11: 1712, 12: 1663, 13: 1666, 14: 1610, 15: 1220, 16: 1604, 17: 1629, 18: 1600, 19: 1564, 20: 1595, 21: 1563, 22: 1555, 23: 1363, 24: 1229, 25: 1745, 26: 1579, 27: 1552, 28: 1507, 29: 1602, 30: 1522, 31: 1494, 32: 1441, 33: 1449, 34: 1399, 35: 1438, 36: 1355, 37: 980, 38: 1423, 39: 1436, 40: 1348, 41: 1403, 42: 1332, 43: 1283, 44: 1199, 45: 1242, 46: 377, 47: 1362, 48: 1382, 49: 1291, 50: 1056, 51: 1011, 52: 935, 53: 1393, 54: 1270, 55: 1186, 56: 1153, 57: 1092, 58: 917, 59: 853, 60: 967, 61: 955, 62: 1530, 63: 1175, 64: 1163}


Collect match data 
Represented by either W, L, D followed by 1 or 2 digits (represnting ID)
Collect list of lists of opponents and show in data frame. Unplayed matches will show up as None

In [483]:
# Collect Match data 
#Declare Opponent ID
OpponentID = []

#Open file 
with open(r"C:\Users\ovadia sutton\Downloads\Project1.txt") as f:
    #Loop through the lines in file 
   for line in f:
    #Collect OpponentID identified as Capital letter followed 2 or 3 spaces followed by 1 or 2 digits followed by "|"
    OpponentID.append(re.findall('[A-Z]{1}\s\s\s?[\d]{1,2}\|', line))
    
#Eliminate empty strings
OpponentID = list(filter(None, OpponentID))

#Create list of lists for clean entries
OpponentIDFinal = []

#Strip undesired elements
for i in OpponentID:
    OpponentIDFinal.append(n.strip("WLD   |"  "")for n in i)

#Convert to int
#Create new list
OpponentIDInt = []

#Loop through OpponentIDFinal
for i in OpponentIDFinal:
    #Save values as int
    OpponentIDInt.append(int(float(x)) for x in i)

#Show OpponentIDInt in Dataframe
df1 = pd.DataFrame(OpponentIDInt)

print(df1)   

     0     1     2     3     4     5     6
0   39  21.0  18.0  14.0   7.0  12.0   4.0
1   63  58.0   4.0  17.0  16.0  20.0   7.0
2    8  61.0  25.0  21.0  11.0  13.0  12.0
3   23  28.0   2.0  26.0   5.0  19.0   1.0
4   45  37.0  12.0  13.0   4.0  14.0  17.0
..  ..   ...   ...   ...   ...   ...   ...
59  33  34.0  45.0  42.0  24.0   NaN   NaN
60  32   3.0  54.0  47.0  42.0  30.0  37.0
61  55   NaN   NaN   NaN   NaN   NaN   NaN
62   2  48.0  49.0  43.0  45.0   NaN   NaN
63  22  30.0  31.0  49.0  46.0  42.0  54.0

[64 rows x 7 columns]


Use OpponentID list of lists as key to identify ratings value from IDRating dictionary then take average 
*We struggled a bit here with reading dictionaries and building new lists to compute average but think we are on the doorstep

In [488]:
#Create list for Opponent Rating
OpponentRating = []

#Loop through OpponentID
for i in OpponentIDFinal:
    #Use as key for OpponentRating
    OpponentRating.append(IDRating[x] for x in i)

#Create list for Opponent Average
"""OpponentAverage = []"""

#Loop through Opponent Rating lists
""""for i in OpponentRating:"""
    #Compute Average of each list and assign to Opponent Average
"""OpponentAverage.append(st.mean(i))"""

#print results 
print(OpponentRating)

[<generator object <genexpr> at 0x0000019939B98EB0>, <generator object <genexpr> at 0x0000019939B98B30>, <generator object <genexpr> at 0x0000019939B98BA0>, <generator object <genexpr> at 0x0000019939B98AC0>, <generator object <genexpr> at 0x0000019939B98A50>, <generator object <genexpr> at 0x0000019939B98970>, <generator object <genexpr> at 0x0000019939B98890>, <generator object <genexpr> at 0x0000019939B986D0>, <generator object <genexpr> at 0x0000019939B98DD0>, <generator object <genexpr> at 0x0000019939A01350>, <generator object <genexpr> at 0x0000019939A012E0>, <generator object <genexpr> at 0x0000019939A01270>, <generator object <genexpr> at 0x0000019939A01200>, <generator object <genexpr> at 0x0000019939A01190>, <generator object <genexpr> at 0x0000019939A01120>, <generator object <genexpr> at 0x0000019939A010B0>, <generator object <genexpr> at 0x0000019939A01040>, <generator object <genexpr> at 0x0000019939B5B200>, <generator object <genexpr> at 0x0000019939B5B270>, <generator 

Create dataframe

In [489]:
#reate data frame with our lists and label columns
df = pd.DataFrame(list(zip(PlayerNamesFinal, PlayerStatesFinal, PlayerScoresFinal, PlayerRatingPreFinal)),
               columns =['Player Name', 'State', 'Score', 'Pre-Rating'])
#print results
print(df)

             Player Name State Score Pre-Rating
0               GARY HUA    ON   6.0       1794
1        DAKSHESH DARURI    MI   6.0       1553
2           ADITYA BAJAJ    MI   6.0       1384
3    PATRICK H SCHILLING    MI   5.5       1716
4             HANSHI ZUO    MI   5.5       1655
..                   ...   ...   ...        ...
59            JULIA SHEN    MI   1.5        967
60         JEZZEL FARKAS    ON   1.5        955
61         ASHWIN BALAJI    MI   1.0       1530
62  THOMAS JOSEPH HOSMER    MI   1.0       1175
63                BEN LI    MI   1.0       1163

[64 rows x 4 columns]


Create CSV file

In [490]:
#Create CSV file with results
df.to_csv(r"C:\Users\ovadia sutton\Downloads\Chess_Results.csv")

Some questions:

1. I was saving lists of lists instead of list of strings and then converting to list of strings as can be observed. Would be simpler to just save as list of strings from the start. When I tried to do that I got a lot of errors. Can use some guidance on this. 

2. How to use a character for criteria but not include in actual list. Here I include the "|" and then strip. How can I use the "|" to define criteria but not include in name/state etc.